In [111]:
#Importing Lib
import pandas as pd
import numpy as np
import numpy_financial as npf
import warnings
from scipy.optimize import newton
from datetime import datetime
from pandas.tseries.offsets import DateOffset


warnings.filterwarnings('ignore')
pd.options.display.max_columns = 200
pd.options.display.max_rows = 500

In [112]:
#Setting IRR 
#folder='79th_buy'
folder='C:/Users/omack/Intrepid/pythonFramework/loan_engine/legacy'
irr_target = 8.05

In [113]:
# #Setting date (running on thursday)

# #for p date1
# from datetime import datetime, timedelta
# today = datetime.today()
# days_until_tuesday = (1 - today.weekday() + 7) % 7  # 1 is Tuesday
# days_until_tuesday = 7 if days_until_tuesday == 0 else days_until_tuesday
# next_tuesday = today + timedelta(days=days_until_tuesday)
# pdate=next_tuesday.strftime('%Y-%m-%d')


# #for sfy and buy
# yestarday=datetime.today() - timedelta(days=1)
# yestarday=yestarday.strftime('%m-%d-%Y')

# #for FX file 
# today = datetime.today()
# first_day_of_current_month = datetime(today.year, today.month, 1)
# last_day_previous_month = first_day_of_current_month - timedelta(days=1)
# last_end= f"{last_day_previous_month.year}_{last_day_previous_month.month:03}_{last_day_previous_month.day:02}"



# print("for pdate=",pdate)
# print("for yestarday=",yestarday)
# print("for fx file=",last_end)

In [114]:
#manual date setting
pdate='2025-11-18'
yestarday='11-12-2025'
last_end='2025_010_31'

In [115]:
#file location and importing

#files_required
# fx3_servicing_file = f"{folder}/files_required/FX3_{last_end}.xlsx"
# fx4_servicing_file = f"{folder}/files_required/FX4_{last_end}.xlsx"
#loans = pd.read_csv(f"{folder}/files_required/Tape20Loans_{(pd.to_datetime('today') - pd.DateOffset(days = 2)).strftime("%m-%d-%Y")}.csv")
loans = pd.read_csv(f"{folder}/files_required/Tape20Loans_10-21-2025.csv")
df_loans_types = pd.read_excel(f"{folder}/files_required/MASTER_SHEET.xlsx")
notes = pd.read_excel(f"{folder}/files_required/MASTER_SHEET - Notes.xlsx")
existing_file_ = pd.read_csv(f"{folder}/files_required/current_assets.csv") #do not change
sfy_file = f"{folder}/files_required/SFY_09-10-2025_ExhibitAtoFormofSaleNotice - Pre-Funding.xlsx" 
prime_file = f"{folder}/files_required/PRIME_09-10-2025_ExhibitAtoFormofSaleNotice - Pre-Funding.xlsx"

#underwriting
underwriting_sfy = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'SFY')
underwriting_prime = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'Prime')
underwriting_sfy_notes = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'SFY - Notes')
underwriting_prime_notes = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = "Prime - Notes")

sfy_comap = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'SFY COMAP')
sfy_comap2 = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'SFY COMAP2')
prime_comap = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'Prime CoMAP')
#prime_comap_new = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'Prime CoMAP - New')

sfy_comap_oct25 = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'SFY COMAP')
sfy_comap_oct25_2 = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'SFY COMAP2')
prime_comap_oct25 = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'Prime CoMAP')
#prime_comap_oct25_2 = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'Prime CoMAP2')

notes_comap = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'Notes CoMAP')

In [116]:
sfy_df = pd.read_excel(sfy_file)
sfy_df = sfy_df.iloc[4:].reset_index(drop=True)
sfy_df.columns = sfy_df.iloc[0]
sfy_df = sfy_df[1:].reset_index(drop=True)
sfy_df.rename(columns={'TU_144': 'TU144','Tu_144': 'TU144','tu_144': 'TU144'}, inplace=True)
print(sfy_df)

0    Count SELLER Loan #        NaN               Asset Type  \
0        1   SFC_6183158  6183158.0           HVAC EQUIPMENT   
1        2   SFC_6186619  6186619.0                POOL DECK   
2        3   SFC_6173826  6173826.0         ROOFING MATERIAL   
3        4   SFC_6124782  6124782.0  HVAC-PLUMBING EQUIPMENT   
4        5   SFC_5637161  5637161.0              PATIO COVER   
...    ...           ...        ...                      ...   
1612  1613   SFC_6195244  6195244.0           HVAC EQUIPMENT   
1613  1614   SFC_6180647  6180647.0           HVAC EQUIPMENT   
1614  1615   SFC_6181609  6181609.0           HVAC EQUIPMENT   
1615  1616   SFC_6177791  6177791.0           HVAC EQUIPMENT   
1616  1617   SFC_6181231  6181231.0                    DOORS   

0             Submit Date Orig. Balance Purchase Price Current Balance    APR  \
0     2025-08-27 00:00:00        100000          95380          100000  12.14   
1     2025-08-28 00:00:00        100000         100420          10000

In [117]:
prime_df = pd.read_excel(prime_file)
prime_df = prime_df.iloc[4:].reset_index(drop=True)
prime_df.columns = prime_df.iloc[0]
prime_df = prime_df[1:].reset_index(drop=True)
print(prime_df)

0    Count SELLER Loan #               Asset Type          Submit Date  \
0        1   SFC_6148883         ROOFING MATERIAL  2025-08-17 00:00:00   
1        2   SFC_6098677         ROOFING MATERIAL  2025-08-01 00:00:00   
2        3   SFC_6129517                   SIDING  2025-08-12 00:00:00   
3        4   SFC_6060290              PATIO COVER  2025-07-22 00:00:00   
4        5   SFC_5764193            WINDOWS/DOORS  2025-04-28 00:00:00   
...    ...           ...                      ...                  ...   
1117  1118   SFC_6194378  HVAC-PLUMBING EQUIPMENT  2025-09-02 00:00:00   
1118  1119   SFC_6184806           HVAC EQUIPMENT  2025-08-28 00:00:00   
1119  1120   SFC_6198484           HVAC EQUIPMENT  2025-09-03 00:00:00   
1120  1121   SFC_6190788       PLUMBING EQUIPMENT  2025-08-30 00:00:00   
1121  1122   SFC_6165728              GARAGE DOOR  2025-08-22 00:00:00   

0    Orig. Balance Purchase Price Current Balance    APR Term  Property City  \
0           100000          860

In [118]:
#df_loans_types
df_loans_types["Platform"] = df_loans_types.platform.str.upper()

#notes
notes['loan program'] = notes['loan program'].apply(lambda x: x + 'notes')
notes["Platform"] = notes.platform.str.upper()

#df_loans_types
df_loans_types = pd.concat([df_loans_types, notes])

#existing_file_
existing_file_['Submit Date'] = pd.to_datetime(existing_file_['Submit Date'])
existing_file_['Purchase_Date'] = pd.to_datetime(existing_file_['Purchase_Date'])
existing_file_['Monthly Payment Date'] = pd.to_datetime(existing_file_['Monthly Payment Date'])
existing_file = existing_file_.copy()
del existing_file_

print(existing_file.shape)
existing_file.Purchase_Date.value_counts().sort_index()

(2833, 73)


Purchase_Date
2025-09-24    728
2025-10-01    546
2025-10-09    595
2025-10-15    476
2025-10-22    488
Name: count, dtype: int64

In [119]:
loans['tagging'] = loans['Loan Group'].apply(lambda x:'SFY' if 'FX3' in x or 'FX1' in x else 'PRIME')
loans['Account Number'] = loans['Account Number'].astype(int)
loans['Status Codes'].fillna("", inplace = True)
loans['Repurchased'] = loans['Status Codes'].apply(lambda x: True if 'REPURCHASE' in [i.strip() for i in x.split(";")] else False)
loans['Open Date'] = pd.to_datetime(loans['Open Date'])
loans['maturityDate'] = pd.to_datetime(loans['maturityDate'])
print(loans['Repurchased'].value_counts())
loans['SELLER Loan #'] = loans['Account Number'].apply(lambda x:"SFC_" + str(x))
repurchased_loans = loans[loans['Repurchased'] == True]['SELLER Loan #'].values

existing_file.reset_index(drop = True, inplace = True)
existing_file.loc[existing_file['SELLER Loan #'].isin(repurchased_loans), 'Repurchase'] = True 

print(repurchased_loans.shape)

Repurchased
False    2342
True        3
Name: count, dtype: int64
(3,)


In [120]:
sfy_df['Platform'] = "SFY"
prime_df['Platform'] = "PRIME"
tuloans = sfy_df[sfy_df.TU144 == 1]['SELLER Loan #'].values
sfy_df.drop(columns = "TU144", inplace = True)
buy_df = pd.concat([prime_df, sfy_df])
buy_df.rename(columns = {"Loan Program" : "loan program"}, inplace = True)
buy_df['loan program'] = buy_df.apply(lambda x: x['loan program'] + 'notes' if x['Application Type'] == 'HD NOTE' else x['loan program'], axis = 1)
buy_df["Repurchase"] = False
buy_df["Repurchase_Date"] = None
buy_df["Purchase_Date"] = pd.to_datetime(pdate)
buy_df["Excess_Asset"] = False
buy_df["Borrowing_Base_eligible"] = True
buy_df["IRR Support Target"] = irr_target
buy_df['Submit Date'] = pd.to_datetime(buy_df['Submit Date'])
buy_df['Purchase_Date'] = pd.to_datetime(buy_df['Purchase_Date'])
buy_df['Monthly Payment Date'] = pd.to_datetime(buy_df['Monthly Payment Date'])
buy_df = buy_df.merge(df_loans_types, on = ['loan program', 'Platform'], how = 'left')
assert sum(buy_df['SELLER Loan #'].duplicated()) == 0

buy_df['Dealer Fee'] = buy_df['Dealer Fee'] / 100
final_df = pd.concat([buy_df, existing_file[existing_file['Purchase_Date'] > '2025-10-01']])
final_df_all = pd.concat([buy_df, existing_file])
buy_df.shape, final_df[final_df['Orig. Balance'] > 50000].shape, final_df.shape, final_df['SELLER Loan #'].nunique(), buy_df[buy_df['tagging'] != 'BD']['Orig. Balance'].sum()

((2739, 74), (35, 74), (4298, 74), 4298, 38395543.39000002)

## Check Purchase Price

In [121]:
existing_file['purchase_price_check'] = existing_file['Lender Price(%)'] == round(existing_file['modeled_purchase_price']*100, 2)
existing_file.shape, existing_file.purchase_price_check.sum()

((2833, 74), np.int64(2832))

In [122]:
buy_df['purchase_price_check'] = buy_df['Lender Price(%)'] == round(buy_df['modeled_purchase_price']*100, 2)
buy_df.shape, buy_df.purchase_price_check.sum()

((2739, 75), np.int64(1147))

In [123]:
final_df['purchase_price_check'] = final_df['Lender Price(%)'] == round(final_df['modeled_purchase_price']*100, 2)
final_df.shape, final_df.purchase_price_check.sum()

((4298, 75), np.int64(2705))

In [124]:
#extra checks as russ mentioned should be close to 0
buy_df['Lender Price(%)_x']=buy_df['Lender Price(%)'].astype(float)/100
buy_df['modeled_price']=buy_df['Lender Price(%)_x']*buy_df['Orig. Balance']
buy_df[['modeled_price','Purchase Price']]
buy_df['modeled_price'].sum()-buy_df['Purchase Price'].sum()

-0.35185883939266205

In [125]:
buy_df['modeled_price']=buy_df['modeled_purchase_price']*buy_df['Orig. Balance'].astype(float)
buy_df[['modeled_price','Purchase Price']]
buy_df['modeled_price'].sum()-buy_df['Purchase Price'].sum()

np.float64(-12099869.59057251)

## Underwriting_checks

In [126]:
# del underwriting_sfy, underwriting_prime

loan_ids = []
min_oncome = []
for row in buy_df[buy_df['Application Type'] != 'HD NOTE'].iterrows():
    if row[1]['platform'] == 'sfy':
        underwriting = underwriting_sfy.copy()
    else:
        underwriting = underwriting_prime.copy()
    if row[1]['SELLER Loan #'] in tuloans or row[1]['SELLER Loan #'] in buy_df[buy_df['Application Type'] == 'HD NOTE']['SELLER Loan #'].values:
        continue
    prog = row[1]['loan program']
    mth_income = row[1]['Income'] / 12
    fico = row[1]['FICO Borrower']
    dti = row[1]['DTI']*100 
    pti = row[1]['PTI']
    reason = ""
    balance = row[1]['Orig. Balance'] - row[1]['Stamp fee']
    # (underwriting.approval_high >= balance) & (underwriting.approval_low <= balance)
    filter_one = underwriting[(underwriting.finance_type_name_nls == prog) & (underwriting.monthly_income_min <= mth_income) & (underwriting.fico_min <= fico)].sort_values("approval_high").reset_index(drop = True)
    meet_crit = False
    for i in filter_one.iterrows():
        if balance <= i[1]['approval_high'] and dti<= i[1]['dti_max']:
            meet_crit = True
    if not meet_crit:
        if fico > 700:
            filter_one = underwriting[(underwriting.finance_type_name_nls == prog) & (underwriting.fico_min <= fico)].sort_values("approval_high").reset_index(drop = True)
            for i in filter_one.iterrows():
                if balance <= i[1]['approval_high'] and dti<= i[1]['dti_max'] and pti <= i[1]["pti_ratio"]:
                    meet_crit = True
            if meet_crit:
                min_oncome.append(row[1]['SELLER Loan #'])
    if not meet_crit:
        loan_ids.append(row[1]['SELLER Loan #'])
    del underwriting
    # break

len(loan_ids), len(min_oncome)

(102, 61)

In [127]:
# del underwriting_sfy_notes, underwriting_prime_notes

loan_ids_notes = []
min_oncome_notes = []
for row in buy_df[buy_df['Application Type'] == 'HD NOTE'].iterrows():
    if row[1]['platform'] == 'sfy':
        underwriting = underwriting_sfy_notes.copy()
    else:
        underwriting = underwriting_prime_notes.copy()
    if row[1]['SELLER Loan #'] in tuloans:
        continue
    prog = row[1]['loan program'].replace('notes', '')
    mth_income = row[1]['Income'] / 12
    fico = row[1]['FICO Borrower']
    dti = row[1]['DTI']*100
    pti = row[1]['PTI']
    reason = ""
    balance = row[1]['Orig. Balance'] - row[1]['Stamp fee']
    (underwriting.approval_high >= balance) & (underwriting.approval_low <= balance)
    filter_one = underwriting[(underwriting.finance_type_name_nls == prog) & (underwriting.monthly_income_min <= mth_income) & (underwriting.fico_min <= fico)].sort_values("approval_high").reset_index(drop = True)
    meet_crit = False
    for i in filter_one.iterrows():
        if balance <= i[1]['approval_high'] and dti <= i[1]['dti_max']:
            meet_crit = True
    if not meet_crit:
        if fico > 700:
            filter_one = underwriting[(underwriting.finance_type_name_nls == prog) & (underwriting.fico_min <= fico)].sort_values("approval_high").reset_index(drop = True)
            for i in filter_one.iterrows():
                if balance <= i[1]['approval_high'] and dti <= i[1]['dti_max'] and pti <= i[1]["pti_ratio"]:
                    meet_crit = True
            if meet_crit:
                min_oncome_notes.append(row[1]['SELLER Loan #'])
    if not meet_crit:
        loan_ids_notes.append(row[1]['SELLER Loan #'])
    del underwriting
    # break

len(loan_ids_notes), len(min_oncome_notes)

(3, 2)

In [128]:
prime_comap_cols_min_fico = {j:int(j[:3]) for i,j in enumerate(['660-699', '700-739', '740-749', '750-769', '770+'])}
prime_comap_cols_min_fico2 = {j:int(j[:3]) for i,j in enumerate(['660-699', '700-739', '740-749', '750+'])}
sfy_comap_cols_min_fico = {j:int(j[:3]) for i,j in enumerate(['660-719', '720-779', '780-799', '800+'])}
sfy_comap_cols_min_fico2 = {j:int(j[:3]) for i,j in enumerate(['660-699', '700-739', '740-749', '750-769','770+'])}
sfy_comap_cols_min_fico3 = {j:int(j[:3]) for i,j in enumerate(['660-719', '720-779', '780+'])}
notes_comap_cols_min_fico = {j:int(j[:3]) for i,j in enumerate(['680-749', '750-769', '770-789', '790+'])}

In [129]:
print(prime_comap_cols_min_fico2)

{'660-699': 660, '700-739': 700, '740-749': 740, '750+': 750}


In [130]:
loan_not_in_comap = []
for row in buy_df[(buy_df['Application Type'] != 'HD NOTE') & (buy_df['purchase_price_check'] == True)].iterrows():
    fico = row[1]['FICO Borrower']
    prog = row[1]['loan program']
    found = False
    if row[1]['platform'] == 'prime':
        if row[1]['Submit Date'] > pd.to_datetime('2025-10-24'):
            if (prog not in prime_comap_oct25[['660-699', '700-739', '740-749', '750+']].stack().unique()) & (prog not in prime_comap_oct25_2[['660-699', '700-739', '740-749', '750-769', '770+']].stack().unique()):
                continue
            else:
                pass
        elif prog not in prime_comap[['660-699', '700-739', '740-749', '750-769', '770+']].stack().unique():
            continue
        else:
            pass
        if row[1]['Submit Date'] > pd.to_datetime('2025-10-24'):
            if prog in prime_comap_oct25[['660-699', '700-739', '740-749', '750+']].stack().unique():
                for col in ['660-699', '700-739', '740-749', '750+']:
                    if prog in prime_comap_oct25[col].values and fico >= prime_comap_cols_min_fico2[col]:
                        found = True
                        break
            elif prog in prime_comap_oct25_2[['660-699', '700-739', '740-749', '750-769', '770+']].stack().unique():
                for col in ['660-699', '700-739', '740-749', '750-769', '770+']:
                    if prog in prime_comap_oct25_2[col].values and fico >= prime_comap_cols_min_fico[col]:
                        found = True
                        break
            else:
                pass
        elif row[1]['Submit Date'] > pd.to_datetime('2024-06-11'):
            for col in ['660-699', '700-739', '740-749', '750-769', '770+']:
                if prog in prime_comap_new[col].values and fico >= prime_comap_cols_min_fico[col]:
                    found = True
                    break
        else:
            for col in ['660-699', '700-739', '740-749', '750-769', '770+']:
                if prog in prime_comap[col].values and fico >= prime_comap_cols_min_fico[col]:
                    found = True
                    break
        if not found:
            loan_not_in_comap.append((row[1]["SELLER Loan #"], prog, "PRIME"))
    else:
        if row[1]['Submit Date'] > pd.to_datetime('2025-10-24'):
            if (prog not in sfy_comap_oct25[['660-719', '720-779', '780+']].stack().unique()) & (prog not in sfy_comap_oct25_2[['660-699', '700-739', '740-749', '750-769','770+']].stack().unique()):
                continue
            else:
                pass
        elif (prog not in sfy_comap[['660-719', '720-779', '780-799', '800+']].stack().unique()) & (prog not in sfy_comap2[['660-699', '700-739', '740-749', '750-769','770+']].stack().unique()):
            continue
        else:
            pass
        if row[1]['Submit Date'] > pd.to_datetime('2025-10-24'):
            if prog in sfy_comap_oct25[['660-719', '720-779', '780+']].stack().unique():
                for col in ['660-719', '720-779', '780+']:
                    if prog in sfy_comap_oct25[col].values and fico >= sfy_comap_cols_min_fico3[col]:
                        found = True
                        break
            elif prog in sfy_comap_oct25_2[['660-699', '700-739', '740-749', '750-769', '770+']].stack().unique():
                for col in ['660-699', '700-739', '740-749', '750-769', '770+']:
                    if prog in sfy_comap_oct25_2[col].values and fico >= sfy_comap_cols_min_fico2[col]:
                        found = True
                        break
            else:
                pass
        elif prog in sfy_comap[['660-719', '720-779', '780-799', '800+']].stack().unique():
            for col in ['660-719', '720-779', '780-799', '800+']:
                if prog in sfy_comap[col].values and fico >= sfy_comap_cols_min_fico[col]:
                    found = True
                    break
        elif prog in sfy_comap2[['660-699', '700-739', '740-749', '750-769','770+']].stack().unique():
            for col in ['660-699', '700-739', '740-749', '750-769','770+']:
                if prog in sfy_comap2[col].values and fico >= sfy_comap_cols_min_fico2[col]:
                    found = True
                    break
        if not found:
            loan_not_in_comap.append((row[1]["SELLER Loan #"], prog, "SFY"))
            
len(loan_not_in_comap)

0

In [131]:
loan_not_in_comap_notes = []
for row in buy_df[(buy_df['Application Type'] == 'HD NOTE') & (buy_df['purchase_price_check'] == True)].iterrows():
    fico = row[1]['FICO Borrower']
    prog = row[1]['loan program']
    found = False
    if prog not in notes_comap[['680-749', '750-769', '770-789', '790+']].stack().unique():
        continue
    for col in ['680-749', '750-769', '770-789', '790+']:
        if prog in notes_comap[col].values and fico >= notes_comap_cols_min_fico[col]:
            found = True
            break
    if not found:
        # reason = f"PRIME loan prog not in fico band - 660-699 - available - {fico}"
        loan_not_in_comap_notes.append((row[1]["SELLER Loan #"], prog, "NOTES"))
            
len(loan_not_in_comap_notes)    

0

In [132]:
#FOR Ourself
final_df[final_df['purchase_price_check'] == False].to_excel(f"{folder}/output/purchase_price_mismatch.xlsx")
buy_df[buy_df["SELLER Loan #"].isin(loan_ids)].to_excel(f"{folder}/output/flagged_loans_first.xlsx")
buy_df[buy_df["SELLER Loan #"].isin(loan_ids_notes)].to_excel(f"{folder}/output/NOTES_flagged_loans_first.xlsx")
buy_df[buy_df["SELLER Loan #"].isin([i[0] for i in loan_not_in_comap])].to_excel(f"{folder}/output/comap_not_passed.xlsx")

In [133]:
#sharing file generation
purchase=final_df[final_df['purchase_price_check'] == False]
flag=buy_df[buy_df["SELLER Loan #"].isin(loan_ids)]
note_flag=buy_df[buy_df["SELLER Loan #"].isin(loan_ids_notes)]
comap=buy_df[buy_df["SELLER Loan #"].isin([i[0] for i in loan_not_in_comap])]
purchase = purchase.iloc[:, :30]
flag = flag.iloc[:, :30]
note_flag = note_flag.iloc[:, :30]
comap = comap.iloc[:, :30]
purchase.to_excel(f"{folder}/output_share/purchase_price_mismatch.xlsx")
flag.to_excel(f"{folder}/output_share/flagged_loans_first.xlsx")
note_flag.to_excel(f"{folder}/output_share/NOTES_flagged_loans_first.xlsx")
comap.to_excel(f"{folder}/output_share/comap_not_passed.xlsx")

## Eligibility Checks

#### Check A

In [134]:
a1=final_df_all[(final_df_all.Repurchase == False) & (final_df_all.platform == 'prime') & (final_df_all.Term <= 144) & (final_df_all.type == 'standard') & (final_df_all['FICO Borrower'] < 700)]['Orig. Balance'].sum() / final_df_all[(final_df_all.Repurchase == False)&(final_df_all.platform == 'prime')]['Orig. Balance'].sum()
print(a1,a1<0.05)

0.023130950018762052 True


In [135]:
# final_df_all[(final_df_all['SELLER Loan #'] != 'SFC_4731527') & (final_df_all.platform == 'prime') & (final_df_all.Term <= 144) & (final_df_all.type == 'standard') & (final_df_all['FICO Borrower'] < 700)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()

In [136]:
# final_df_all[(final_df_all.platform == 'prime') & (final_df_all.Term <= 144) & (final_df_all.type == 'standard') & (final_df_all['FICO Borrower'] < 700)]['Orig. Balance'].sum()

#### Check B

In [137]:
b1=final_df_all[(final_df_all.Repurchase == False) & (final_df_all.platform == 'prime') & (final_df_all.Term > 144) & (final_df_all.type == 'standard') & (final_df_all['FICO Borrower'] < 700)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()
print(b1,b1<0.03)

0.00209497932246601 True


In [138]:
# b2=final_df_all[(final_df_all.platform == 'prime') & (final_df_all.Term > 144) & (final_df_all.type == 'standard') & (final_df_all['FICO Borrower'] < 700)]['Orig. Balance'].sum() 
# print(b2)

In [139]:
b3=final_df_all[(final_df_all.platform == 'prime') & (final_df_all.Term > 144) & (final_df_all.type == 'standard') & (final_df_all['FICO Borrower'] < 700)]['Orig. Balance'].shape[0] / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].shape[0]
print(b3,b3<0.03)

0.001932367149758454 True


#### Check C

In [140]:
c1=final_df_all[(final_df_all.platform == 'prime') & (final_df_all.Term > 144) & (final_df_all.type == 'standard') & (final_df_all['FICO Borrower'] >= 700)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()
print(c1,c1<0.35)

0.2735426024099911 True


#### Check D

In [141]:
d1=final_df_all[(final_df_all.platform == 'prime') & (final_df_all.type == 'hybrid')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()
print(d1,d1<0.35)

0.2633663624437587 True


#### Check E

In [142]:
e1=final_df_all[(final_df_all.platform == 'prime') & (final_df_all.type == 'ninp')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()
print(e1,e1<0.15)

0.04691762082000882 True


#### Check F

In [143]:
f1=final_df_all[(final_df_all.platform == 'prime') & (final_df_all.type == 'epni')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()
print(f1,f1<0.18)

0.04431647346674706 True


#### Check G

In [144]:
g1=final_df_all[(final_df_all.platform == 'prime') & (final_df_all.type == 'wpdi')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()
print(g1,g1<0.15)

0.16745489132255384 False


#### Check H

In [145]:
#Ignore
h1=final_df_all[(final_df_all.platform == 'prime')]['Lender Price(%)'].max()
print(h1,h1<101.87)

101.73 True


In [146]:
h2=final_df_all[(final_df_all.platform == 'prime') & (final_df_all['Lender Price(%)'] > 100) & (final_df_all['Lender Price(%)'] <= 103)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()
print(h2,h2<0.35)

0.15862962928376015 True


In [147]:
h3=np.sum((final_df_all['Dealer Fee'] * final_df_all['Orig. Balance'])/final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum())
print(h3,h3<0.15)

0.11574597963185362 True


#### Check I

In [148]:
i1=final_df_all[(final_df_all.platform == 'prime') & (final_df_all['Orig. Balance'] > 50000)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()
print(i1,i1<0.38)

0.0446048895490377 True


In [149]:
i2=final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].mean()
print(i2,i2<20000)

14116.448000000004 True


#### Check J

In [150]:
(pd.pivot_table(final_df_all[(final_df_all.platform == 'prime')], values = 'Orig. Balance', aggfunc = 'sum', columns='Property State') / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()).T.sort_values("Orig. Balance", ascending = False)

,Orig. Balance
Property State,
CA,0.1396
FL,0.134868
TX,0.118066
PA,0.051747
OH,0.046781
AZ,0.043801
MI,0.035104
NC,0.030471
MD,0.027826


#### Check L

In [151]:
l1=final_df_all[(final_df_all.platform == 'prime') & (final_df_all['FICO Borrower'] < 680)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()
print(l1,l1<0.5)

0.01306492629427777 True


In [152]:
l2=final_df_all[(final_df_all.platform == 'prime') & (final_df_all['FICO Borrower'] < 700)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()
print(l2,l2<0.7)

0.10414386905808702 True


In [153]:
l3=np.sum((final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'] * final_df_all[(final_df_all.platform == 'prime')]['FICO Borrower'] ) / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum())
print(l3,l3>700)

735.7608153693485 True


In [154]:
final_df_all[(final_df_all.platform == 'prime')]['FICO Borrower'].mean()

np.float64(732.0009661835749)

### SFY

#### Check A

In [155]:
a1=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'hybrid')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(a1,a1<0.85)

0.15075783213773464 True


In [156]:
a2=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'hybrid') & (final_df_all.APR < 7.0)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(a2,a2<0.25)

0.0 True


#### Check B

In [157]:
b1=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'ninp')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(b1,b1<0.3)

0.2541410845138058 True


In [158]:
b2=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'ninp') & (final_df_all.promo_term > 6)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(b2,b2<0.27)

0.2541410845138058 True


In [159]:
b3=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'ninp') & (final_df_all.promo_term > 12)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(b3,b3<0.15)

0.0327352934812456 True


In [160]:
b4=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'ninp') & (final_df_all.Term > 84)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(b4,b4<=0)

0.0 True


#### Check C

In [161]:
c1=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'epni')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(c1,c1<=0.25)

0.0296633555768267 True


#### Check D

In [162]:
d1=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'wpdi')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(d1,d1<=0.17)

0.19014800185879074 False


In [163]:
d2=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type.isin([ 'wpdi', 'wpdi_bd']))]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(d2,d2<=0.17)

0.19082706631366356 False


In [164]:
d3=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'wpdi') & (final_df_all.promo_term >= 12)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(d3,d3<=0.09)

0.14577517189949074 False


In [165]:
d4=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type.isin([ 'wpdi', 'wpdi_bd'])) & (final_df_all.promo_term >= 12)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(d4,d4<=0.09)

0.14577517189949074 False


#### Check E

In [166]:
e1=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'standard') & (final_df_all.Term > 120)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(e1,e1<=0.3)

0.039389243831434405 True


In [167]:
e2=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type.isin([ 'standard', 'standard_bd'])) & (final_df_all.Term > 120)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(e2,e2<=0.3)

0.039389243831434405 True


In [168]:
e3=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'standard') & (final_df_all.Term > 144)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(e3,e3<=0.28)

0.039389243831434405 True


In [169]:
e4=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type.isin([ 'standard', 'standard_bd'])) & (final_df_all.Term > 144)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(e4,d4<=0.28)

0.039389243831434405 True


#### Check F

In [170]:
f1=final_df_all[(final_df_all.platform == 'sfy')]['Lender Price(%)'].max()
print(f1,f1<=101.21)

101.07 True


In [171]:
f2=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all['Lender Price(%)'] > 100) & (final_df_all['Lender Price(%)'] <= 103)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(f2,f2<=0.4)

0.30230943364085017 True


In [172]:
f3=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all['Lender Price(%)'] > 100) & (final_df_all['Lender Price(%)'] <= 103) & (final_df_all['loan program'] != "Unsec Std - 999 - 120")]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(f3,f3<=0.37)

0.191835945878869 True


In [173]:
f4=np.sum(final_df_all['Dealer Fee'] * final_df_all['Orig. Balance'])/final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(f4,f4<=0.15)

0.09985845797661598 True


#### Check G

In [174]:
g1=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all['Orig. Balance'] > 50000)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(g1,g1<=0.38)

0.06410578019194553 True


In [175]:
g2=final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].mean()
print(g2,g2<=20000)

12512.053191725146 True


#### Check H

In [176]:
(pd.pivot_table(final_df_all[(final_df_all.platform == 'sfy')], values = 'Orig. Balance', aggfunc = 'sum', columns='Property State') / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()).T.sort_values("Orig. Balance", ascending = False)

,Orig. Balance
Property State,
CA,0.121301
FL,0.102494
TX,0.091787
PA,0.056132
AZ,0.038648
IL,0.036835
NY,0.035697
OH,0.035671
NC,0.034116


#### Check J

In [177]:
j1=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all['FICO Borrower'] < 680)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(j1,j1<=0.5)

0.04635440440211934 True


In [178]:
j2=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all['FICO Borrower'] < 700)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(j2,j2<=0.7)

0.12758595247339677 True


In [179]:
j3=np.sum((final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'] * final_df_all[(final_df_all.platform == 'sfy')]['FICO Borrower'] ) / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum())
print(j3,j3>=700)

774.1996102457023 True


In [180]:
j4=final_df_all[(final_df_all.platform == 'sfy')]['FICO Borrower'].mean()
print(j4,j4>=700)

764.1001108237901 True


#### Check L

In [181]:
l1=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'wpdi_bd')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(l1,l1<=0.01)

0.0006790644548727617 True


In [182]:
l2=final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'standard_bd')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(l2)

0.032911983985685574


In [183]:
l3=final_df_all[(final_df_all.type.isin( ['standard_bd', 'wpdi_bd']))]['Purchase Price'].sum() / final_df_all['Purchase Price'].sum()
print(l3)

0.014600322022018304


In [184]:
l4=final_df_all[(final_df_all.type.isin( ['standard_bd', 'wpdi_bd'])) & (final_df_all.Excess_Asset != True)]['Purchase Price'].sum() / final_df_all['Purchase Price'].sum()
print(l4)

0.014600322022018304


In [185]:
l5=buy_df[(buy_df.platform == 'sfy') & (buy_df.type == 'standard_bd')]['Orig. Balance'].sum() / buy_df[(buy_df.platform == 'sfy')]['Orig. Balance'].sum()
print(l5)

0.04773549770783458


In [186]:
#special asset
s1=final_df_all[(final_df_all.new_programs == True)&(final_df_all.Repurchase == False) & (final_df_all.platform == 'prime')]['Orig. Balance'].sum() / final_df_all[(final_df_all.Repurchase == False)&(final_df_all.platform == 'prime')]['Orig. Balance'].sum()
print(s1,s1<0.02)

0.07431872396411329 False


In [187]:
s1=final_df_all[(final_df_all.new_programs == True)&(final_df_all.Repurchase == False) & (final_df_all.platform == 'sfy')]['Orig. Balance'].sum() / final_df_all[(final_df_all.Repurchase == False)&(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()
print(s1,s1<0.02)

0.026781863668126498 False


In [188]:
#--------------------------------------------------End--------------------------------------